# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Imported the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray <file.path>\establishments.json`

In [36]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [37]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [38]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'auto_db',
 'config',
 'epa',
 'fruit_db',
 'local',
 'mets',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [39]:
# assign the uk_food database to a variable name
db = mongo["uk_food"]

In [40]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [41]:
# review a document in the establishments collection + save as variable
establishments_c = db["establishments"]
establishments_c.find_one()

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [42]:
# Create a dictionary for the new restaurant data
new_restaurant = {"BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True}

In [43]:
# Insert the new restaurant into the collection
establishments_c.insert_one(new_restaurant)

In [44]:
# Check that the new restaurant was inserted
list(establishments_c.find({"BusinessName":"Penang Flavours"}))

[{'_id': ObjectId('63ef22c78652b2618016c97f'),
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'PostCode': 'SE18 7DY',
  'Phone': '',
  'LocalAuthorityCode': '511',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
  'SchemeType': 'FHRS',
  'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
  'RightToReply': '',
  'Distance': 4623.972328074718,
  'NewRatingPending': True}]

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [45]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1}

pprint((establishments_c.find_one(query,fields)))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 '_id': ObjectId('63ef22c78652b2618016c97f')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [46]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName":"Penang Flavours"}
update = {"$set": {"BusinessTypeID": 1}}

establishments_c.update_one(query, update)

In [47]:
# Confirm that the new restaurant was updated
fields = {"BusinessName":1, "BusinessTypeID": 1, "BusinessType": 1}
pprint(list(establishments_c.find({"BusinessName":"Penang Flavours"}, fields)))

[{'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('63ef22c78652b2618016c97f')}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [55]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
print(establishments_c.count_documents(query))

994


In [56]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments_c.delete_many(query)

In [57]:
# Check if any remaining documents include Dover
print(establishments_c.count_documents(query))

0


In [58]:
# Check that other documents remain with 'find_one'
pprint(establishments_c.find_one())

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63ef22c78652b2618016c97f'),
 'geocode': {'latitude': '51.49014200', 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [69]:
# Change the data type from String to Decimal for longitude
query = {}
update = [{'$set': {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]

establishments_c.update_many(query, update)

In [70]:
# Change the data type from String to Decimal for latitude
query = {}
update = [{'$set': {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]

establishments_c.update_many(query, update)

In [74]:
# Check that the coordinates are now numbers - searching by type double -> if it's a string it won't appear
query = {"geocode.longitude": {"$type": "double"}, "geocode.latitude": { "$type": "double"}}
fields = {"geocode.longitude":1,"geocode.latitude":1}
pprint(list(establishments_c.find(query, fields).limit(5)))


[{'_id': ObjectId('63ef22c78652b2618016c97f'),
  'geocode': {'latitude': 51.490142, 'longitude': 0.08384}},
 {'_id': ObjectId('63ef22e381e5f0fbe1b62fe5'),
  'geocode': {'latitude': 51.083812, 'longitude': 1.195625}},
 {'_id': ObjectId('63ef22e381e5f0fbe1b62fe6'),
  'geocode': {'latitude': 51.086058, 'longitude': 1.196408}},
 {'_id': ObjectId('63ef22e381e5f0fbe1b62fe7'),
  'geocode': {'latitude': 51.085797, 'longitude': 1.194762}},
 {'_id': ObjectId('63ef22e381e5f0fbe1b62fea'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}]
